Reading the CSV file.
Truncating the data to 10747 rows.
Handling missing values by filling them with zeros.
Converting specific columns to float data type.


#Standardizing selected columns using StandardScaler from sklearn.preprocessing.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

print(data.tail(1))

             date    open    high     low   close      volume  adjusted_close  \
10746  2023-07-31  196.06  196.49  195.26  196.45  38824100.0        196.1851   

       change_percent  avg_vol_20d  
10746            0.32   49803320.0  


Covert date to torch: date_embeddings

In [3]:



import torch
import torch.nn as nn
import pandas as pd

# Sample data with a 'date' column containing dates from 1980-12-12 to 2023-07-31
data2 = pd.DataFrame({'date': pd.date_range(start='1980-12-12', end='2023-07-31')})

# Convert 'date' column to datetime
data2['date'] = pd.to_datetime(data['date'])

# Extract day, month, and year from the date column
data['day'] = data2['date'].dt.day
data['month'] = data2['date'].dt.month
data['year'] = data2['date'].dt.year

# Define the embedding dimensions
embedding_dim = 1  # You can adjust this dimension as needed

# Create embedding layers for day, month, and year
day_embedding = nn.Embedding(32, embedding_dim)  # 0-31 days
month_embedding = nn.Embedding(13, embedding_dim)  # 1-12 months
year_embedding = nn.Embedding(44, embedding_dim)  # Embedding for years from 1980 to 2023

# Convert day, month, and year to tensors with Long data type
day_tensor = torch.LongTensor(data['day'].values)
month_tensor = torch.LongTensor(data['month'].values)
year_tensor = torch.LongTensor(data['year'].values - 1980)  # Convert years to an index from 0 to 43


# Pass tensors through embedding layers to get embeddings
day_embeddings = day_embedding(day_tensor)
month_embeddings = month_embedding(month_tensor)
year_embeddings = year_embedding(year_tensor)

# Concatenate the embeddings
date_embeddings = torch.cat((day_embeddings, month_embeddings, year_embeddings), dim=1)

# Print the resulting embeddings



Concatenate Date Embeddings - Features
Split
Scale

In [4]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Concatenate date embeddings with data
x_feature_tensors = torch.tensor(x_data, dtype=torch.float32)
x_combined = torch.cat((x_feature_tensors, date_embeddings), dim=1)

y_feature_tensors = torch.tensor(y_data, dtype=torch.float32)
y_combined = torch.cat((y_feature_tensors, date_embeddings), dim=1)

# Split the data into training and testing sets
train_size = int(len(x_combined) * 0.67)
x_train, x_test = x_combined[:train_size], x_combined[train_size:]
y_train, y_test = y_combined[:train_size], y_combined[train_size:]

# Initialize the scaler
scaler = MinMaxScaler()

# Fit the scaler only on the training data and transform both training and test data
# fitting the MinMaxScaler to your training data using fit_transform and then
# applying the same transformation (based on the statistics learned) to your test data using transform
x_train_scaled = scaler.fit_transform(x_train.detach().numpy())
x_test_scaled = scaler.transform(x_test.detach().numpy())  # Use transform, not fit_transform

y_train_scaled = scaler.fit_transform(y_train.detach().numpy())
y_test_scaled = scaler.transform(y_test.detach().numpy())  # Use transform, not fit_transform

# Convert back to tensors if needed
x_train_scaled = torch.tensor(x_train_scaled, dtype=torch.float32)
x_test_scaled = torch.tensor(x_test_scaled, dtype=torch.float32)

y_train_scaled = torch.tensor(y_train_scaled, dtype=torch.float32)
y_test_scaled = torch.tensor(y_test_scaled, dtype=torch.float32)




LSTM Model Definition
Initialize the model, loss, and optimizer
Training using Walk-Forward Validation
Backward pass and optimize
Evaluate the model on the test set


In [5]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

# Adjustable Parameters
learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 2
batch_evaluation_frequency = 10
epochs = len(x_train_scaled) - window_size
batch_size = 1
input_size = x_train_scaled.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

# LSTM Model Definition
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(x.device)

        out, (hidden, cell) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model, loss, and optimizer
model = LSTMModel(input_size, hidden_dim, n_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training using Walk-Forward Validation
for i in range(window_size, epochs):
    optimizer.zero_grad()

    # Extract training window with batch size
    start_idx = i - window_size
    end_idx = i
    x_window = torch.tensor(x_train_scaled[start_idx:end_idx], dtype=torch.float32)
    y_window = torch.tensor(y_train_scaled[start_idx:end_idx], dtype=torch.float32)

    # Reshape x_window to match LSTM input shape [batch_size=1, sequence_length=window_size, input_size=input_size]
    x_window = x_window.view(1, window_size, input_size)

    # Forward pass
    outputs = model(x_window)
    loss = criterion(outputs, y_window)

    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    # Evaluate the model on the test set
    if i % batch_evaluation_frequency == 0:
        with torch.no_grad():
            x_test_window = torch.tensor(x_test_scaled[:test_window_size], dtype=torch.float32)
            y_test_window = torch.tensor(y_test_scaled[:test_window_size], dtype=torch.float32)

            # Reshape x_test_window to match LSTM input shape [batch_size=1, sequence_length=test_window_size, input_size=input_size]
            x_test_window = x_test_window.view(1, test_window_size, input_size)

            test_outputs = model(x_test_window)
            test_loss = criterion(test_outputs, y_test_window)
            print(f"Iteration {i}, Train Loss: {loss.item()}, Test Loss: {test_loss.item()}")


C:\Users\pitr7\AppData\Local\Temp\ipykernel_3284\1154295661.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train_scaled[start_idx:end_idx], dtype=torch.float32)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_3284\1154295661.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_window = torch.tensor(y_train_scaled[start_idx:end_idx], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([50, 4])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  retur

Iteration 50, Train Loss: 0.1081152930855751, Test Loss: 0.259853333234787
Iteration 60, Train Loss: 0.1157444417476654, Test Loss: 0.1605028212070465
Iteration 70, Train Loss: 0.13963651657104492, Test Loss: 0.1759324073791504
Iteration 80, Train Loss: 0.1515057384967804, Test Loss: 0.12344531714916229
Iteration 90, Train Loss: 0.14506983757019043, Test Loss: 0.13202723860740662
Iteration 100, Train Loss: 0.13277091085910797, Test Loss: 0.13700547814369202
Iteration 110, Train Loss: 0.11867053806781769, Test Loss: 0.15371064841747284
Iteration 120, Train Loss: 0.09899081289768219, Test Loss: 0.16038472950458527
Iteration 130, Train Loss: 0.08744464814662933, Test Loss: 0.19435039162635803
Iteration 140, Train Loss: 0.0831906795501709, Test Loss: 0.2209063023328781
Iteration 150, Train Loss: 0.0960044115781784, Test Loss: 0.24022653698921204
Iteration 160, Train Loss: 0.10042592883110046, Test Loss: 0.2109508514404297
Iteration 170, Train Loss: 0.11298396438360214, Test Loss: 0.2161919

In [7]:
import torch

# Set the model to evaluation mode
model.eval()

# Function to initialize the hidden state (customize based on your model architecture)
def init_hidden(batch_size):
    return (torch.zeros(1, batch_size, 50), torch.zeros(1, batch_size, 50))  # Assuming 50 hidden units

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_dim)

# Initialize the hidden state
hidden = init_hidden(batch_size=1)

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value and update the hidden state
        forecasted_value, hidden = model(input_sequence, hidden)

        # Reshape the forecasted value to have the same sequence length as input_sequence
        forecasted_value = forecasted_value.view(1, 1, input_dim)

        # Update the input sequence by removing the first element and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


NameError: name 'input_dim' is not defined

In [ ]:
import torch

# Set the model to evaluation mode
model.eval()

# Function to initialize the hidden state
def init_hidden(batch_size):
    # Initialize with zeros
    return (torch.zeros(1, batch_size, 50), torch.zeros(1, batch_size, 50))  # Assuming 50 hidden units

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_dim)

# Initialize the hidden state
hidden = init_hidden(batch_size=1)

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value and update the hidden state
        forecasted_value, hidden = model(input_sequence, hidden)  # Assuming your model's forward() method takes input and hidden

        # Reshape the forecasted value to have the same sequence length as input_sequence
        forecasted_value = forecasted_value.view(1, 1, input_dim)

        # Update the input sequence by removing the first element and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


TypeError: forward() takes 2 positional arguments but 3 were given

In [ ]:
import torch

# Set the model to evaluation mode
model.eval()

# Function to initialize the hidden state (customize based on your model architecture)
def init_hidden(batch_size):
    # For simplicity, assuming it's a single LSTM layer
    # Initialize with zeros
    return (torch.zeros(1, batch_size, 50), torch.zeros(1, batch_size, 50))  # Assuming 50 hidden units

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_dim)

# Initialize the hidden state
hidden = init_hidden(batch_size=1)

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value and update the hidden state
        forecasted_value, hidden = model(input_sequence, hidden)

        # Reshape the forecasted value to have the same sequence length as input_sequence
        forecasted_value = forecasted_value.view(1, 1, input_dim)

        # Update the input sequence by removing the first element and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


TypeError: forward() takes 2 positional arguments but 3 were given

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define the number of future steps you want to forecast
num_forecast_steps = 1

# Initialize the input sequence for forecasting
input_sequence = x_train_scaled[-window_size:].view(1, window_size, input_dim)  # Assuming 'device' is set properly

# Forecast the next 'num_forecast_steps' values
for _ in range(num_forecast_steps):
    with torch.no_grad():
        # Predict the next value
        forecasted_value = model(input_sequence)

        # Reshape the forecasted value for concatenation
        forecasted_value = forecasted_value.view(1, 1, 1)

        # Update the input sequence by removing the first element and adding the forecasted value at the end
        input_sequence = torch.cat((input_sequence[:, 1:, :], forecasted_value), dim=1)

# Convert the forecasted value back to the original scale using the scaler
forecasted_value_unscaled = scaler.inverse_transform(forecasted_value.cpu().numpy())

print(f"Forecasted value of 'close' for the next time step: {forecasted_value_unscaled[0][0][0]}")


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 10 but got size 1 for tensor number 1 in the list.

In [ ]:
print(input_sequence.shape)
print(forecasted_value.shape)

torch.Size([1, 50, 10])
torch.Size([1, 1, 1])
